# Attributes of Winning Teams
## Exploring the Data of the EU Soccer League

The goal of this analysis will be to see if there are common attributes amoung the top performing leagues in the European Soccer League. Data is from a Kaggle post,[European Soccer Database](https://www.kaggle.com/hugomathien/soccer) by Hugomathien.

The process of discovery will be the following
1. Create SQL Queries to gather/organize original data
    * Which teams the players belong too (most recent year)
    * Join the player attributes to those teams (most recent performance scores)
    * Which teams placed top on the league (most recent year)
2. Read the SQL Queries into Pandas for further manipulation
3. Provide exploratory visualization showing the combined attributes of each top-performing team
4. Make a judgement if there are common attributes that matter most to success

## Setting Up Environment

First I want to make sure I have my enviroments set up. I have a work station with three monitors but I am not always there so I want to make sure I can do analysis on the fly on my Chromebook. To do this I have set up a cloud instance with [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). I started the project on my workstation and then createe my [GitHub Repo](https://github.com/GarrettMarkScott/European_Soccer_Exploring). In the process of doing this I quickly learned that the 300mb database (SQLite) was too large for a simple git push command. This forced me to create a .gitignore file that I placed the database in. I then was able to clone the git repo into my Jupyter Cloud Instance and simply upload the SQLite database into the same directory in my cloud instance. Now I can push/pull from both machines without large file transfers.

I used the below code to check to make sure my environment was working.

In [19]:
#These are the general items for simple loading

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import itertools
import pprint

pp = pprint.PrettyPrinter(indent=4)

conn = sqlite3.connect('soccer_data.sqlite')
tables = pd.read_sql('SELECT * FROM sqlite_master WHERE type="table";', conn)
countries = pd.read_sql('SELECT * FROM Country;', conn)
players = pd.read_sql('SELECT player_api_id, player_name FROM Player;', conn)
player_att = pd.read_sql('SELECT * FROM Player_Attributes;',conn)
league_teams = pd.read_sql('SELECT team_long_name FROM Team', conn)
player_att.columns.tolist() #I use this line for previewing original data

['id',
 'player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

## Matching Players to Teams

Next was time to create the SQL queries to organize the raw data a little bit so it was easier to work with. I was able to borrow my girlfriends mac and load the SQLite datebase into [DB Browser for SQLite] for quick SQL Queries (since chromebook can't run programs).

I spent quite a bit of time trying to figure out how to merge the players to teams. As seen in the screenshot below there was not a common key between player and team. My assumption is that this is due to players switching various teams over their career.

![Table Screenshot](https://i.imgur.com/KCpqgS5.jpg)

I checked the *player_fifa_api_id* but that had 11,060 DISTINCT rows. I was able to create the query below that used the Match table data to see which players were associated with each match. Due to there being substitutions throughout the match, it is assumed that these 11 players are the starting roster for each team during the 2007-2008 season (May-August). 

This turned out to be a pretty big challege because I did not relize DB Browser for SQLite seemingly does not have a formal Date type but rather stores date types as a *Timestring* format. If anyone has any information regarding this and how to deal with it in the best/cleanest way I have made a [Stack Overflow Post](https://stackoverflow.com/questions/56202599/converting-timestrings-to-datetime-db-browser)

In [20]:
#Matches from most recent season and the starting players (in ID format)
#Is there an easy solution to use parameters for date range?
matches = pd.read_sql_query('''SELECT t.team_long_name, m.home_player_1, m.home_player_2, 
                                      m.home_player_3,m.home_player_4, m.home_player_5,
                                      m.home_player_6, m.home_player_7, m.home_player_8,
                                      m.home_player_9, m.home_player_10, m.home_player_11,
                                      m.date, m.home_team_goal, m.away_team_goal 
                                 FROM Team t 
                                 JOIN Match m ON t.team_api_id = m.home_team_api_id 
                                WHERE DATE(m.date) BETWEEN "2015-05-01" and "2016-05-25" 
                                ORDER BY t.team_long_name;''', conn, )
matches

,team_long_name,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,date,home_team_goal,away_team_goal
0,1. FC Köln,212815.0,36395.0,36934.0,264221.0,307210.0,36086.0,167589.0,127945.0,459493.0,166449.0,196366.0,2015-05-10 00:00:00,2,0
1,1. FC Köln,212815.0,36395.0,36934.0,264221.0,22824.0,450976.0,36086.0,127945.0,459493.0,196366.0,166449.0,2015-05-23 00:00:00,2,2
2,1. FC Köln,212815.0,127945.0,231753.0,303800.0,307210.0,166449.0,36086.0,167589.0,459493.0,177941.0,71605.0,2015-10-31 00:00:00,0,0
3,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,166449.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-11-21 00:00:00,0,0
4,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,238438.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-12-05 00:00:00,0,1
5,1. FC Köln,212815.0,212377.0,231753.0,36934.0,303800.0,307210.0,450976.0,36086.0,22824.0,127945.0,238438.0,2015-12-19 00:00:00,2,1
6,1. FC Köln,212815.0,212377.0,36934.0,303800.0,307210.0,167589.0,36086.0,127945.0,166449.0,260470.0,71605.0,2016-01-23 00:00:00,1,3
7,1. FC Köln,212815.0,212377.0,231753.0,303800.0,307210.0,127945.0,167589.0,36086.0,260470.0,238438.0,71605.0,2015-08-22 00:00:00,1,1
8,1. FC Köln,212815.0,36934.0,28435.0,303800.0,127945.0,450976.0,307210.0,259439.0,238438.0,260470.0,71605.0,2016-02-13 00:00:00,3,1
9,1. FC Köln,212815.0,36934.0,28435.0,303800.0,127945.0,36086.0,307210.0,259439.0,450976.0,71605.0,260470.0,2016-02-26 00:00:00,0,1


## Create a Function to Connect Player ID to Player Name
To keep our code modular as possible the function below can be used to return a player's name based on ID input.

The first result **player_api_id** is the test and you will see that it returns **Aaron Appindangoye** correctly.

In [21]:
players

,player_api_id,player_name
0,505942,Aaron Appindangoye
1,155782,Aaron Cresswell
2,162549,Aaron Doran
3,30572,Aaron Galindo
4,23780,Aaron Hughes
5,27316,Aaron Hunt
6,564793,Aaron Kuhl
7,30895,Aaron Lennon
8,528212,Aaron Lennox
9,101042,Aaron Meijers


In [22]:
#Reusable function to translate api key to player name

def player_id_to_name(id): 
    return(players[players['player_api_id'] == id].iloc[0,1])

player_id_to_name(505942)

'Aaron Appindangoye'

## Creating Compound Dictionary of Teams, Players, & Attributes

Build a for loop that builds a teams dictionary with team as the key and players as the values.

This is our goal:
```
teams = {"team 1": {"Player 1": {"Dribbling": 4,
                                 "Attack": 5,
                                 "Speed": 9},
                    "Player 2": {"Dribbling": 6,
                                 "Attack": 10,
                                 "Speed": 3},
                    "Player 3": {"Dribbling": 2,
                                 "Attack": 5,
                                 "Speed": 8},
                    },
         "team 2": {"Player 1": {"Dribbling": 9,
                                 "Attack": 6,
                                 "Speed": 2},
                    "Player 2": {"Dribbling": 10,
                                 "Attack": 2,
                                 "Speed": 8},
                    "Player 3": {"Dribbling": 5,
                                 "Attack": 5,
                                 "Speed": 5},
                    }
            }
```

### Creating dictionary of teams and starting players from most recent season

#### Step 1: Create a list of teams

In [23]:
#Create a list of the teams to iterate
team_names = league_teams.values.tolist()
merged_team_names = list(itertools.chain(*team_names)) #Flatten list of lists
team_names = merged_team_names
print(len(team_names))
pprint.pprint(team_names)

299
['KRC Genk',
 'Beerschot AC',
 'SV Zulte-Waregem',
 'Sporting Lokeren',
 'KSV Cercle Brugge',
 'RSC Anderlecht',
 'KAA Gent',
 'RAEC Mons',
 'FCV Dender EH',
 'Standard de Liège',
 'KV Mechelen',
 'Club Brugge KV',
 'KSV Roeselare',
 'KV Kortrijk',
 'Tubize',
 'Royal Excel Mouscron',
 'KVC Westerlo',
 'Sporting Charleroi',
 'Sint-Truidense VV',
 'Lierse SK',
 'KAS Eupen',
 'Oud-Heverlee Leuven',
 'Waasland-Beveren',
 'KV Oostende',
 'Royal Excel Mouscron',
 'Manchester United',
 'Newcastle United',
 'Arsenal',
 'West Bromwich Albion',
 'Sunderland',
 'Liverpool',
 'West Ham United',
 'Wigan Athletic',
 'Aston Villa',
 'Manchester City',
 'Everton',
 'Blackburn Rovers',
 'Middlesbrough',
 'Tottenham Hotspur',
 'Bolton Wanderers',
 'Stoke City',
 'Hull City',
 'Fulham',
 'Chelsea',
 'Portsmouth',
 'Birmingham City',
 'Wolverhampton Wanderers',
 'Burnley',
 'Blackpool',
 'Swansea City',
 'Queens Park Rangers',
 'Norwich City',
 'Southampton',
 'Reading',
 'Crystal Palace',
 'Cardiff C

#### Step 2: Create a function to get each starting player of a team
*Note the original SQL query already constrained the data to most recent season*

In [24]:
def get_team_players(x):
    #Gather all the players that have started in the match data. 
    all_team_players = matches[matches['team_long_name'] == x].iloc[:,2:12].values.astype(int).tolist()
    merged_team_players = list(itertools.chain(*all_team_players)) #flattens list of lists
    all_team_players = [] #resets variable for append function below

    #this for loops takes all the players api numbers, flattens to unique numbers, and stores them in variable
    for i in merged_team_players:  
        name = i
        if name not in all_team_players:
            all_team_players.append(name)
        else:
            pass
    return(all_team_players)

print(get_team_players('1. FC Köln'))

[36395, 36934, 264221, 307210, 36086, 167589, 127945, 459493, 166449, 196366, 22824, 450976, 231753, 303800, 177941, 71605, 260470, 238438, 212377, 28435, 259439]


#### Step 3: Create a dictionary using players as the values of each team
**Goal:** To create the example dictionary below

In [25]:
teams = {}
teams['blue'] = {'stevie': {"Dribbling": 4,
                            "Attack": 5,
                            "Speed": 9},
                 'robbie': {"Dribbling": 6,
                            "Attack": 10,
                            "Speed": 3},
                 'joey':   {"Dribbling": 2,
                            "Attack": 5,
                            "Speed": 8}
                }

print('Original Goal')                  
pp.pprint(teams)
print()


Original Goal
{   'blue': {   'joey': {'Attack': 5, 'Dribbling': 2, 'Speed': 8},
                'robbie': {'Attack': 10, 'Dribbling': 6, 'Speed': 3},
                'stevie': {'Attack': 5, 'Dribbling': 4, 'Speed': 9}}}



The code below is how I achieved the goal.

A roadblock I ran into was that my team_names variable was returning single item lists that needed to be converted into a string for itterating.

In [26]:
#print('The output of team_names[0] is: '+ str(team_names[0])+'\n')
#print('The type of team_names[0] is: '+ str(type(str(team_names)))) #It was a LIST!!!! Now string
#print('The output of get_team_players("KRC Genk") is: \n')
#print(get_team_players('KRC Genk'))

teams = {}
for i in range(len(team_names)):   #Currently only returns first 5
    try:
        player_apis = get_team_players(str(team_names[i])) #Had to convert from list
        player_names = {}
        for api in player_apis:
            player_name = player_id_to_name(api)
            player_names[player_name] = player_att[player_att['player_api_id'] == api].iloc[1,:].transform(dict)
        teams[team_names[i]] = player_names
    except:
        #print('Team had no players registered')
        pass

#pp.pprint(teams) 

**Use the cell below to test different dictionary lookups.**

In [27]:
teams['Hull City']["Dame N'Doye"]

{'id': 33945,
 'player_fifa_api_id': 179875,
 'player_api_id': 101969,
 'date': '2016-01-21 00:00:00',
 'overall_rating': 77.0,
 'potential': 77.0,
 'preferred_foot': 'right',
 'attacking_work_rate': 'high',
 'defensive_work_rate': 'medium',
 'crossing': 56.0,
 'finishing': 75.0,
 'heading_accuracy': 77.0,
 'short_passing': 63.0,
 'volleys': 71.0,
 'dribbling': 74.0,
 'curve': 69.0,
 'free_kick_accuracy': 70.0,
 'long_passing': 38.0,
 'ball_control': 74.0,
 'acceleration': 78.0,
 'sprint_speed': 78.0,
 'agility': 74.0,
 'reactions': 75.0,
 'balance': 56.0,
 'shot_power': 85.0,
 'jumping': 70.0,
 'stamina': 79.0,
 'strength': 84.0,
 'long_shots': 74.0,
 'aggression': 72.0,
 'interceptions': 40.0,
 'positioning': 79.0,
 'vision': 64.0,
 'penalties': 60.0,
 'marking': 35.0,
 'standing_tackle': 38.0,
 'sliding_tackle': 19.0,
 'gk_diving': 14.0,
 'gk_handling': 13.0,
 'gk_kicking': 9.0,
 'gk_positioning': 8.0,
 'gk_reflexes': 16.0}

# Picking Top Teams
We now have a nice dictionary of the teams, their registered players, and their attributes. Now let's write a command to discover the team with the most wins. Let's use some visualization to help us determine if we want data from the top 3, top 5, etc. out of the 299 teams.

This should be based on winning percent vs. total sum.

In [15]:
matches['final_status'] = 'NaN'
matches.head()

,team_long_name,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,date,home_team_goal,away_team_goal,points_awarded,final_staus,final_status
0,1. FC Köln,212815.0,36395.0,36934.0,264221.0,307210.0,36086.0,167589.0,127945.0,459493.0,166449.0,196366.0,2015-05-10 00:00:00,2,0,0,a,NaN
1,1. FC Köln,212815.0,36395.0,36934.0,264221.0,22824.0,450976.0,36086.0,127945.0,459493.0,196366.0,166449.0,2015-05-23 00:00:00,2,2,0,a,NaN
2,1. FC Köln,212815.0,127945.0,231753.0,303800.0,307210.0,166449.0,36086.0,167589.0,459493.0,177941.0,71605.0,2015-10-31 00:00:00,0,0,0,a,NaN
3,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,166449.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-11-21 00:00:00,0,0,0,a,NaN
4,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,238438.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-12-05 00:00:00,0,1,0,a,NaN


The code below appends additional columns. `'final_status'` uses the goal data to determine who won. `'points_awarded'` then delivers a point system for 'win' and 'tie'. This number will be used to determine the top teams.

In [39]:
matches['final_status'] = "NaN"
matches['points_awarded'] = 0
matches.loc[matches['home_team_goal'] > matches['away_team_goal'], ['final_status','points_awarded']] = ['win',2]
matches.loc[matches['home_team_goal'] == matches['away_team_goal'], ['final_status','points_awarded']] = ['tie',1]
matches.loc[matches['home_team_goal'] < matches['away_team_goal'], 'final_status'] = 'lose'
matches.head(25)

,team_long_name,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,date,home_team_goal,away_team_goal,final_status,points_awarded
0,1. FC Köln,212815.0,36395.0,36934.0,264221.0,307210.0,36086.0,167589.0,127945.0,459493.0,166449.0,196366.0,2015-05-10 00:00:00,2,0,win,2
1,1. FC Köln,212815.0,36395.0,36934.0,264221.0,22824.0,450976.0,36086.0,127945.0,459493.0,196366.0,166449.0,2015-05-23 00:00:00,2,2,tie,1
2,1. FC Köln,212815.0,127945.0,231753.0,303800.0,307210.0,166449.0,36086.0,167589.0,459493.0,177941.0,71605.0,2015-10-31 00:00:00,0,0,tie,1
3,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,166449.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-11-21 00:00:00,0,0,tie,1
4,1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,238438.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-12-05 00:00:00,0,1,lose,0
5,1. FC Köln,212815.0,212377.0,231753.0,36934.0,303800.0,307210.0,450976.0,36086.0,22824.0,127945.0,238438.0,2015-12-19 00:00:00,2,1,win,2
6,1. FC Köln,212815.0,212377.0,36934.0,303800.0,307210.0,167589.0,36086.0,127945.0,166449.0,260470.0,71605.0,2016-01-23 00:00:00,1,3,lose,0
7,1. FC Köln,212815.0,212377.0,231753.0,303800.0,307210.0,127945.0,167589.0,36086.0,260470.0,238438.0,71605.0,2015-08-22 00:00:00,1,1,tie,1
8,1. FC Köln,212815.0,36934.0,28435.0,303800.0,127945.0,450976.0,307210.0,259439.0,238438.0,260470.0,71605.0,2016-02-13 00:00:00,3,1,win,2
9,1. FC Köln,212815.0,36934.0,28435.0,303800.0,127945.0,36086.0,307210.0,259439.0,450976.0,71605.0,260470.0,2016-02-26 00:00:00,0,1,lose,0


Now that we have the points for each team. The next step is to figure out how many games they each played.

In [40]:
data = {'team_name': matches.groupby(['team_long_name']),
        'games_played': matches.groupby(['team_long_name'])['date'].count().values.tolist(),
       }
matches.groupby(['team_long_name'])

### Create Team Points

In [59]:
team_points = matches.groupby(['team_long_name']).sum()[['points_awarded']]
#team_points

### Create Games Played

In [60]:
team_games_played = matches.groupby(['team_long_name']).count()[['date']]
team_games_played.rename(columns={'date':'games_played'}, inplace=True) #proper rename
#team_games_played

### Create Performance Score Column

This is a custom agregation that originally assined 2 points/win and 1 point/tie. Some teams played more games than others in the season so using a total sum would not be accurate. The underlaying of `performance_score` is the percent of `'points_awarded' / 'games_played'` then simplifies the data by transforming it into an number.

In [74]:
team_totals = team_games_played.merge(team_points, on=('team_long_name'))
team_totals['performance_score'] = ((team_totals['points_awarded'] / team_totals['games_played'])*100).astype(int)
team_totals.sort_values('performance_score', ascending=False)

,games_played,points_awarded,performance_score
team_long_name,,,
FC Dordrecht,1,2,200
Queens Park Rangers,1,2,200
St. Mirren,1,2,200
Club Brugge KV,15,28,186
Napoli,21,39,185
Borussia Dortmund,19,35,184
SL Benfica,19,34,178
Paris Saint-Germain,21,37,176
Juventus,21,37,176


Some of the teams have very little recorded data so they will not be considered for our top teams. As a rather barbaric quality test I googled a couple of the top teams to see if they are still top placers (since data is from 2016). An example is that Napoli is currently 2nd in the Italian league (05/25/19). I will consider our `'performance_score'` number as an accurate indicator and continue moving forward.

Two key variables that have been created are:
* Teams (with players & attributes)
* Team_Totals (indicating top teams)

In [82]:
teams['Napoli']['Christian Maggio']['crossing']

79.0

To access the nested directory keys we need to provide a key to access the nested keys. To avoid a brittle code that relies on hard-coding a teams nameHowever, because not all teams have registered players in this database there is no garuntee that the first returned team will have a players dictionary to dive into. To combat this I am using the code below that states a team needs to have 10 registered players to be valid. We will then use that. Even though we are dealing with an unordered dictionary this code will work for our purposes no matter the order.

**Syntax Goal:** list(teams\[valid_team]\[any player on team].keys())

In [163]:
valid_team_index = 0
while team_player_count < 10: #11 Players on a team
    for i in range(len(teams.keys())):
        team_player_count = len(teams[list(teams.keys())[i]])
        if team_player_count > 10:
            valid_team_index = i
            break
            
valid_team = list(teams.keys())[valid_team_index]
pp.pprint(teams[valid_team])

{   'Alejandro Pozuelo': {   'acceleration': 77.0,
                             'aggression': 60.0,
                             'agility': 76.0,
                             'attacking_work_rate': 'high',
                             'balance': 82.0,
                             'ball_control': 74.0,
                             'crossing': 63.0,
                             'curve': 72.0,
                             'date': '2016-04-28 00:00:00',
                             'defensive_work_rate': 'medium',
                             'dribbling': 76.0,
                             'finishing': 60.0,
                             'free_kick_accuracy': 59.0,
                             'gk_diving': 10.0,
                             'gk_handling': 8.0,
                             'gk_kicking': 14.0,
                             'gk_positioning': 10.0,
                             'gk_reflexes': 8.0,
                             'heading_accuracy': 39.0,
                            

    'Sebastien Dewaest': {   'acceleration': 50.0,
                             'aggression': 81.0,
                             'agility': 41.0,
                             'attacking_work_rate': 'medium',
                             'balance': 38.0,
                             'ball_control': 56.0,
                             'crossing': 39.0,
                             'curve': 33.0,
                             'date': '2016-04-21 00:00:00',
                             'defensive_work_rate': 'high',
                             'dribbling': 28.0,
                             'finishing': 33.0,
                             'free_kick_accuracy': 39.0,
                             'gk_diving': 16.0,
                             'gk_handling': 7.0,
                             'gk_kicking': 12.0,
                             'gk_positioning': 16.0,
                             'gk_reflexes': 10.0,
                             'heading_accuracy': 77.0,
                           

In [162]:
attribute_list = list(teams[valid_team][list(teams[valid_team].keys())[1]])
print(attribute_list)

['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']


In [160]:
#get team averages definition
def average_team_attributes(team)
    

71.0
49.0
47.0
66.0
48.0
75.0
76.0
59.0
66.0
57.0
39.0
70.0
72.0
71.0
63.0
63.0
70.0
67.0
70.0
63.0
55.0
66.0


In [170]:
team_stats = pd.DataFrame.from_dict(teams[valid_team])

In [176]:
team_stats = team_stats.T

In [178]:
team_stats.mean(axis = 0)

acceleration              72.681818
aggression                65.227273
agility                   69.409091
balance                   68.227273
ball_control              68.818182
crossing                  62.863636
curve                     57.727273
dribbling                 65.818182
finishing                 53.636364
free_kick_accuracy        56.636364
gk_diving                 10.863636
gk_handling                9.590909
gk_kicking                10.363636
gk_positioning            10.818182
gk_reflexes               11.272727
heading_accuracy          62.045455
id                    110106.363636
interceptions             53.136364
jumping                   70.318182
long_passing              62.363636
long_shots                58.545455
marking                   50.136364
overall_rating            70.409091
penalties                 57.454545
player_api_id         273261.000000
player_fifa_api_id    195140.181818
positioning               59.681818
potential                 75

The next section of this will be to make a function that returns the team's average attributes. We will then chart the top 5 teams and visually look for common attribute combinations.